In [37]:
pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adam


2025-10-13 09:48:43.397879: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-13 09:48:43.570306: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-13 09:48:48.770977: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-13 09:48:58.032997: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [3]:
def load_dataset(folders, label):
    images = []
    labels = []
    for folder in folders:
        for file in os.listdir(folder):
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_path = os.path.join(folder, file)
                img = load_img(img_path, target_size=(28,28), color_mode='grayscale')
                arr = img_to_array(img)/255.0
                images.append(arr)
                labels.append(label)
    return images, labels

In [ ]:

background_folders = ["/home/ubuntu/Downloads/padhai-text-non-text-classification-level-1/level_1_train/level_1/background"]
text_folders = [
    "/home/ubuntu/Downloads/padhai-text-non-text-classification-level-1/level_1_train/level_1/en",
    "/home/ubuntu/Downloads/padhai-text-non-text-classification-level-1/level_1_train/level_1/hi",
    "/home/ubuntu/Downloads/padhai-text-non-text-classification-level-1/level_1_train/level_1/ta"
]


In [ ]:
from sklearn.model_selection import train_test_split

X_bg, y_bg = load_dataset(background_folders, 0)
X_txt, y_txt = load_dataset(text_folders, 1)


X_total = np.array(X_bg + X_txt)
y_total = np.array(y_bg + y_txt)
X_total = X_total.reshape(X_total.shape[0], -1)


X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=42, shuffle=True)

In [21]:
def build_model(input_dim):
    model = Sequential([
        Dense(512, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
def train_and_evaluate(optimizer, name, epochs=20, batch_size=32):
    model = build_model(X_train.shape[1])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    
    
    y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
    

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    

    print(f"\nOptimizer: {name}")
    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)
    
    
    df = pd.DataFrame({"ImageID": np.arange(len(y_pred)), "Prediction": y_pred})
    df.to_excel(f"predictions_{name}.xlsx", index=False)
    
    return {"Optimizer": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1}


In [23]:
optimizers = [
    (SGD(learning_rate=0.01), 'SGD'),
    (SGD(learning_rate=0.01, momentum=0.9), 'Momentum'),
    (SGD(learning_rate=0.01, momentum=0.9, nesterov=True), 'Nesterov'),
    (Adagrad(learning_rate=0.01), 'Adagrad'),
    (RMSprop(learning_rate=0.001), 'RMSprop'),
    (Adam(learning_rate=0.001), 'Adam')
]


In [24]:
optimizer_classes = [
    (SGD, {}),
    (SGD, {"momentum":0.9}),
    (SGD, {"momentum":0.9, "nesterov":True}),
    (Adagrad, {}),
    (RMSprop, {}),
    (Adam, {})
]

results = []

for opt_class, params in optimizer_classes:
    optimizer_instance = opt_class(**params)  
    name = f"{opt_class.__name__}{params if params else ''}"
    metrics = train_and_evaluate(optimizer_instance, name)
    results.append(metrics)


/home/ubuntu/miniconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/miniconda3/lib/python3.13/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

Optimizer: SGD
Accuracy: 0.9888, Precision: 1.0000, Recall: 0.9765, F1: 0.9881
Confusion Matrix:
[[94  0]
 [ 2 83]]


/home/ubuntu/miniconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/miniconda3/lib/python3.13/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

Optimizer: SGD{'momentum': 0.9}
Accuracy: 0.9721, Precision: 1.0000, Recall: 0.9412, F1: 0.9697
Confusion Matrix:
[[94  0]
 [ 5 80]]


/home/ubuntu/miniconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/miniconda3/lib/python3.13/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


...

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step

Optimizer: Adagrad
Accuracy: 0.9777, Precision: 1.0000, Recall: 0.9529, F1: 0.9759
Confusion Matrix:
[[94  0]
 [ 4 81]]


/home/ubuntu/miniconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/miniconda3/lib/python3.13/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

Optimizer: RMSprop
Accuracy: 0.9665, Precision: 1.0000, Recall: 0.9294, F1: 0.9634
Confusion Matrix:
[[94  0]
 [ 6 79]]


/home/ubuntu/miniconda3/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/ubuntu/miniconda3/lib/python3.13/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step

Optimizer: Adam
Accuracy: 0.9721, Precision: 1.0000, Recall: 0.9412, F1: 0.9697
Confusion Matrix:
[[94  0]
 [ 5 80]]


In [20]:
results_df = pd.DataFrame(results)
print("\nAll Optimizers Results:")
print(results_df)
results_df.to_excel("optimizer_comparison_results.xlsx", index=False)


All Optimizers Results:
                                Optimizer  Accuracy  Precision    Recall  \
0                                     SGD  0.983240        1.0  0.964706   
1                    SGD{'momentum': 0.9}  0.983240        1.0  0.964706   
2  SGD{'momentum': 0.9, 'nesterov': True}  0.983240        1.0  0.964706   
3                                 Adagrad  0.972067        1.0  0.941176   
4                                 RMSprop  1.000000        1.0  1.000000   
5                                    Adam  0.983240        1.0  0.964706   

         F1  
0  0.982036  
1  0.982036  
2  0.982036  
3  0.969697  
4  1.000000  
5  0.982036  
